# DoD Contracts - Top Contract Winning Vendors

## Aims
The aim of this notebook is to identify the top vendors in terms of contract wins and contract value. In this specific context, we will be looking at the US Department of Defense.

In [59]:
import pandas as pd
import requests

def get_awards(year, limit=100):

    start_date = f"{year}-01-01"
    end_date = f"{year + 1}-01-01"

    # Define the API endpoint URL
    base_url = "https://api.usaspending.gov/api/v2/search/spending_by_award/"

    url = f"{base_url}"

    agency_object = {
        "type": "awarding",  
        "tier": "toptier",   
        "name": "Department of Defense"  
    }
    
    # Construct the payload
    payload =   {
      "subawards": False,
      "limit": limit,
      "filters": {
          "award_type_codes": ["A", "B", "C"],
          "time_period": [{"start_date": start_date, "end_date": end_date}],
          "agencies": [agency_object]
      },
      "fields": [
          "Award ID",
          "Recipient Name",
          "Start Date",
          "End Date",
          "Award Amount",
          "Awarding Agency",
          "Awarding Sub Agency",
          "Contract Award Type",
          "Award Type",
          "Funding Agency",
          "Funding Sub Agency"
      ]
    }

    # Make the API request
    response = requests.post(url, json=payload)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return None
    


In [58]:
def get_award_count(year):

    start_date = f"{year}-01-01"
    end_date = f"{year + 1}-01-01"  
    # Define the API endpoint URL
    base_url = "https://api.usaspending.gov/api/v2/search/spending_by_award_count/"

    url = f"{base_url}"

    agency_object = {
        "type": "awarding",  
        "tier": "toptier",   
        "name": "Department of Defense"  
    }

    time_period_object = {
        "start_date": start_date,
        "end_date": end_date
    }
    
    # Construct the payload
    payload =   {
      "filters": {
          "award_type_codes": ["A", "B", "C"],
          "time_period": [time_period_object],
          "agencies": [agency_object]
      }
    }

    # Make the API request
    response = requests.post(url, json=payload)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return None
    


In [60]:
year = 2008

data = get_awards(2008, 100)
award_count = get_award_count(2008)
award_count

{'results': {'contracts': 1175500,
  'direct_payments': 0,
  'grants': 0,
  'idvs': 0,
  'loans': 0,
  'other': 0},
 'messages': ['For searches, time period start and end dates are currently limited to an earliest date of 2007-10-01.  For data going back to 2000-10-01, use either the Custom Award Download feature on the website or one of our download or bulk_download API endpoints as listed on https://api.usaspending.gov/docs/endpoints.']}

In [64]:
def get_award_counts(start_year, end_year):
    award_count = {}
    for year in range(start_year, end_year + 1):
        count_data = get_award_count(year)
        award_count[year] = count_data['results']['contracts']
    return award_count


In [65]:
award_counts = get_award_counts(2008, 2023)
award_counts

{2008: 1175500,
 2009: 1131993,
 2010: 1134855,
 2011: 1108574,
 2012: 1069271,
 2013: 979278,
 2014: 1535124,
 2015: 2829387,
 2016: 3342674,
 2017: 3522348,
 2018: 4141117,
 2019: 3867089,
 2020: 3674508,
 2021: 3956571,
 2022: 3923725,
 2023: 2360601}

In [39]:
df = pd.DataFrame.from_dict(data['results'])

In [40]:
df.head()

,internal_id,Award ID,Recipient Name,Start Date,End Date,Award Amount,Awarding Agency,Awarding Sub Agency,Contract Award Type,Award Type,Funding Agency,Funding Sub Agency,awarding_agency_id,agency_slug,generated_internal_id
0,38972555,ZZ98,AMERISOURCEBERGEN DRUG CORP,2008-06-18,2008-06-18,2737.0,Department of Defense,Defense Logistics Agency,DO,None,Department of Defense,Defense Logistics Agency,1173,department-of-defense,CONT_AWD_ZZ98_9700_SPM20005D2003_9700
1,38972535,ZZ96,AMERISOURCEBERGEN DRUG CORP,2008-06-17,2008-06-17,23824.0,Department of Defense,Defense Logistics Agency,DO,None,Department of Defense,Defense Logistics Agency,1173,department-of-defense,CONT_AWD_ZZ96_9700_SPM20005D2003_9700
2,38972511,ZZ93,AMERISOURCEBERGEN DRUG CORP,2008-06-14,2008-06-16,3655.0,Department of Defense,Defense Logistics Agency,DO,None,Department of Defense,Defense Logistics Agency,1173,department-of-defense,CONT_AWD_ZZ93_9700_SPM20005D2003_9700
3,38972492,ZZ91,AMERISOURCEBERGEN DRUG CORP,2008-06-14,2008-06-16,20938.0,Department of Defense,Defense Logistics Agency,DO,None,Department of Defense,Defense Logistics Agency,1173,department-of-defense,CONT_AWD_ZZ91_9700_SPM20005D2003_9700
4,38972471,ZZ88,AMERISOURCEBERGEN DRUG CORP,2008-06-13,2008-06-13,28674.0,Department of Defense,Defense Logistics Agency,DO,None,Department of Defense,Defense Logistics Agency,1173,department-of-defense,CONT_AWD_ZZ88_9700_SPM20005D2003_9700


In [43]:
award_count

{'results': {'contracts': 1175500,
  'direct_payments': 0,
  'grants': 0,
  'idvs': 0,
  'loans': 0,
  'other': 0},
 'messages': ['For searches, time period start and end dates are currently limited to an earliest date of 2007-10-01.  For data going back to 2000-10-01, use either the Custom Award Download feature on the website or one of our download or bulk_download API endpoints as listed on https://api.usaspending.gov/docs/endpoints.']}